In [ ]:
import os
import csv
from sklearn.utils import shuffle
samples = []
with open('../data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2) 
import cv2
import numpy as np
import sklearn

def generator(samples, batch_size=128):
    num_samples = len(samples)
    while 1:
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            #correction = 0.15
            images = []
            angles = []
            for batch_sample in batch_samples:
                filename = batch_sample[0].split('\\')[-1]
                name = '..\\data\\IMG\\'+filename
                bgr = cv2.imread(name)
                image = bgr[...,::-1] # convert the bgr image read from cv2 to rgb format
                images.append(image)
                angle = float(batch_sample[3])
                angles.append(angle)
                        
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train,y_train)


In [ ]:
train_generator = generator(train_samples) #create a generator for training data set 
validation_generator = generator(validation_samples) #create a generator for validation data set 


In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
import keras
model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape = (160,320,3), output_shape = (160,320,3))) # normalization
model.add(Cropping2D(cropping=((65,30),(0,0))))
model.add(Conv2D(24,(5,5),subsample=(2,2),activation="relu"))
model.add(Conv2D(36,(5,5),subsample=(2,2),activation="relu"))
model.add(Conv2D(48,(5,5),subsample=(2,2),activation="relu"))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(Dropout(0.5)) # dropout rate set to be 0.5
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))
Adam = keras.optimizers.Adam(lr=0.0003) 
model.compile(loss='mse',optimizer=Adam)
model.fit_generator(train_generator, steps_per_epoch= len(train_samples),
validation_data=validation_generator, validation_steps=len(validation_samples), epochs=3, verbose = 1)

model.save('model2.h5')
print('model saved')
